# 1. Synthetic Dataset Generation

## Type Definitions

Following are some useful type definitions to avoid verbose code later on

In [1]:
from typing import Dict
import networkx as nx
from typing import Tuple

# Type definitions
Ged = int
HistoryKey = int
HistoryValue = Tuple[nx.Graph, Ged]
HistoryEntry = Tuple[HistoryKey, HistoryValue]
History = Dict[HistoryKey, HistoryValue]

## RandomGraphGenerator

RandomGraphGenerator is a class used to randomly generate graphs with some input parameters allowing for some customization

In [2]:
from random import randint
from random import uniform
from random import choice
import networkx as nx

class RandomGraphGenerator():
    """RandomGraphGenerator can be used to generate random graph with parameters in input.\n
    The way it does so ensures that there are no isolates nodes (there should be at least 2 nodes in the generated graph)"""
    def generate_random_ER_graph(self, nmin=2, nmax=10, pmin=0.2, pmax=1):
        """Erdős-Rényi graph generation.\n
        Parameters:
        1. nmin, min number of nodes: >= 2
        2. nmax, max number of nodes: >= nmin
        3. pmin, min probability for edge creation: >= 0.01
        4. pmax, max probability for edge creation: <= 1
        """
        n  = randint(nmin, nmax) # Number of nodes
        p = uniform(pmin, pmax) # Probability for edge creation
        G = nx.gnp_random_graph(n, p, seed=None, directed=False)
        return self._make_connected(G)

    def generate_random_BA_graph(self, nmin=2, nmax=10):
        """Erdős-Rényi graph generation.\n
        Parameters:
        1. nmin, min number of nodes: nmin >= 2
        2. nmax, max number of nodes: >= nmin
        """
        n  = randint(nmin, nmax) # Number of nodes
        m = randint(1, n-1) # Number of edges to attach from a new node to existing nodes,
        if not (m >= 1 and m < n):
            raise Exception(f"m >= 1 and m < n", f"m={m}", f"n={n}")
        G = nx.barabasi_albert_graph(n, m, seed=None, initial_graph=None)
        return self._make_connected(G)
    
    def _make_connected(self, G : nx.Graph):
        for node in list(nx.isolates(G)):
            target_node = choice([n for n in G.nodes() if n != node])
            G.add_edge(node, target_node)
        return G

## Consecutor Archetype

### Consecutor

Consecutor is an abstract class which defines the prototypes to generate a graph G' starting from G

In [3]:
from abc import ABC, abstractmethod
import networkx as nx
from typing import List
from random import randint
from random import random
from copy import deepcopy

class UnprocessableError(Exception):
    """Error raised when no further process can be applied (useful in some Consecutor)"""
    pass

class Consecutor(ABC):
    """Abstract Consecutor with base utility methods ready for the concrete Consecutor.
    \nUsed to generate a Graph G' from G."""
    def next(self, G : nx.Graph) -> HistoryValue:
        """Return a tuple with a new graph G' and the distance from G (can be 0)"""
        if not self._is_processable(G):
            raise UnprocessableError()
        copy = deepcopy(G)
        rand = random()
        return self._next(copy, rand)
    
    @abstractmethod
    def _next(self, G : nx.Graph, rand : float) -> HistoryValue:
        """Actual next logic from concrete classes"""
        raise NotImplementedError()
    
    def _is_processable(self, G : nx.Graph) -> bool:
        """Whether you can make a 'next' on graph G"""
        return len(self._nodes(G)) > 0 and len(self._edges(G)) > 0
    
    def _nodes(self, G : nx.Graph) -> List[int]:
        """Return the list of nodes of the graph G"""
        return list(G.nodes)
    
    def _edges(self, G : nx.Graph) -> List[Tuple[int, int]]:
        """Return the list of edges of the graph G"""
        return list(G.edges)
    
    def _rand_obj_list(self, l : List) -> object | None:
        """Return a random object in list if not empty else None"""
        return l[randint(0, len(l) - 1)] if len(l) > 0 else None
    
    def _new_node(self, G : nx.Graph) -> int:
        """Return a new node for the graph G (biggest indexed node + 1)"""
        return (self._nodes(G)[-1] + 1) if len(self._nodes(G)) > 0 else 0
    
    def _rand_node(self, G : nx.Graph) -> int | None:
        """Return a random existing node of G if any else None"""
        return self._rand_obj_list(self._nodes(G))
    
    def _new_edge(self, G : nx.Graph) -> Tuple[int, int] | None:
        """Return a new edgre for the graph G if not fully-connected else None"""
        return self._rand_obj_list(list(nx.non_edges(G)))
    
    def _rand_edge(self, G : nx.Graph) -> Tuple[int, int] | None:
        """Return a random existing edge of G if any else None"""
        return self._rand_obj_list(self._edges(G))

### Incremental Consecutor

IncrementalConsecutor is a class that generates G' starting from G by adding something to it

In [4]:
class IncrementalConsecutor(Consecutor):
    """IncrementalConsecutor add nodes and edges. The way it does so ensures there are no isolates at any moment."""
    def _next(self, G : nx.Graph, rand : float) -> HistoryValue:
        if rand <= 0.33:
            return self.__add_node_and_edges(G)
        else:
            return self.__add_edge(G)
    
    def __add_node_and_edges(self, G : nx.Graph) -> HistoryValue:
        """Add a new node and k edges from the new node to random nodes"""
        new_node = super()._new_node(G)
        G.add_node(new_node)
        nodes = super()._nodes(G)
        k = randint(1, len(nodes) - 1)
        choices = list(filter(lambda n : n != new_node, nodes))
        for _ in range(0, k):
            target = super()._rand_obj_list(choices)
            G.add_edge(new_node, target)
            choices.remove(target)
        return G, (1+k)
            
    def __add_edge(self, G : nx.Graph) -> HistoryValue:
        """Add a new edge if not fully connected"""
        new_edge = super()._new_edge(G)
        if new_edge is None:
            return G, 0
        G.add_edge(*new_edge)
        return G, 1
        

### DecrementalConsecutor

DecrementalConsecutor is a class that generates G' starting from G by removing something from it

In [5]:
class DecrementalConsecutor(Consecutor):
    """DecrementalConsecutor removes nodes and edges, after any atomic operation it also removes isolated nodes.
    \nThis is due to how data is stored (edge adj matrix which drops isolates informations)."""
    def _next(self, G : nx.Graph, rand : float) -> HistoryValue:
        if rand <= 0.33:
            return self._remove_edge(G)
        else:
            return self._remove_node_and_edges(G)
        
    def _remove_node_and_edges(self, G : nx.Graph) -> HistoryValue:
        """Remove a random node along with its edges, if this causes a node to be isolated it is removed aswell"""
        rvm_node = self._rand_node(G)
        if rvm_node is None:
            return G, 0
        degree = G.degree(rvm_node)
        G.remove_node(rvm_node)
        isolated = list(nx.isolates(G))
        G.remove_nodes_from(isolated)
        return G, (1+degree+len(isolated))
            
    def _remove_edge(self, G : nx.Graph) -> HistoryValue:
        """Remove a random edge if there are any, if this causes a node to be isolated it is removed aswell"""
        rvm_edge = self._rand_edge(G)
        if rvm_edge is None:
            return G, 0
        G.remove_edge(*rvm_edge)
        isolated = list(nx.isolates(G))
        G.remove_nodes_from(isolated)
        return G, (1+len(isolated))
        

## ConsecutorExecutor

ConsecutorExecutor is a class that manages that execution of a concatenations of generations of new graphs by exploiting Consecutor class

In [6]:
from typing import Callable

class ConsecutorExecutor():
    """ConsecutorExecutor can be used to execute steps consecutions starting from a graph G"""
    def __init__(self, consecutor: Consecutor):
        self.consecutor = consecutor
    
    def execute(self, 
                G : nx.Graph, 
                steps = 100, 
                stopper : Callable[[nx.Graph], bool] = None,
                skip_zero_ged = True,
                ) -> History :
        """Perform steps attempts to modify graph G.
        Parameters:
        1. G, the graph where to start from
        2. steps, the number of atomic modifications
        3. stopper, an early custom stopping function on newly generated graph
        4. skip_zero_ged, a Consecutor may return a G' with ged 0 w.r.t. G
        Returns a dict representing the history of graph generations with edit distance from previous graph.
        """
        history = {}
        history[-1] = (G, 0)
        for i in range(0, steps):
            try:
                # Generation and update G
                G, ged = self.consecutor.next(G)
            except UnprocessableError:
                break
            # Custom stopping condition on newly generated graph
            if stopper is not None and stopper(G):
                break
            # Save only when necessary
            if ged != 0 or not skip_zero_ged:
                history[i] = (G, ged)
        return history

## Plotting Functions

Just two functions used together to either plot two graphs side by side or to plot a series of graphs side by side by an history

In [7]:
import matplotlib.pyplot as plt

def plot_graphs_sbs(g1 : nx.Graph, g2 : nx.Graph):
    """Utility function to plot two graphs side by side"""
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    nx.draw(g1, with_labels=True, font_weight='bold', ax=axes[0])
    nx.draw(g2, with_labels=True, font_weight='bold', ax=axes[1])
    plt.tight_layout()
    plt.show()

def plot_cons_hist_entries(history: History, stop=None):
    """Utility function to plot entries of history in a consecutive manner"""
    entries = list(history.items())
    for i,e in enumerate(entries):
        n = entries[i+1]
        plot_graphs_sbs(e[1][0], n[1][0])
        if stop is not None and i+1 == stop:
            break
        if i+1 == len(entries) - 1:
            break 

## HistoryUtilities

HistoryUtilities is a class that provides useful functions for history datatype.

In [8]:
import pandas as pd
from itertools import combinations

class HistoryUtilities():
    """HistoryUtilities is responsible for providing common history utilities functions."""
        
    def build_combinations_df(self, history : History):
        """Function that generate a dataframe out from history"""
        entries = list(history.items())
        all_combs = list(combinations(entries, 2))
        df = pd.DataFrame({
            "graph_1": list(map(lambda c : list(c[0][1][0].edges), all_combs)),
            "graph_2": list(map(lambda c : list(c[1][1][0].edges), all_combs)),
            "ged": list(map(lambda c : self.calculate_ged_comb(history, c), all_combs)),
            "labels_1": list(map(lambda c : ["1"]*(max(list(c[0][1][0].nodes) or [-1])+1), all_combs)),
            "labels_2": list(map(lambda c : ["1"]*(max(list(c[1][1][0].nodes) or [-1])+1), all_combs)),
        })
        return df
        
    def assert_geds(self, history : History):
        """Checks for every combination of history the real ged distance"""
        all_entries = history.items()
        all_combinations = list(combinations(all_entries, 2))
        for comb in all_combinations:
            g1 = comb[0][1][0]
            g2 = comb[1][1][0]
            r = nx.graph_edit_distance(g1, g2)
            e = self.calculate_ged_comb(history, comb)
            assert r == e
            
    def calculate_ged_comb(self, history : History, comb : Tuple[HistoryEntry, HistoryEntry]):
        """Returns the artificial ged distance given an entry combination"""
        delimiters = [comb[0][0], comb[1][0]]
        delimiters.sort()
        min, max = delimiters[0], delimiters[1]
        sum = 0
        for entry in history.items():
            key = entry[0]
            value = entry[1]
            if min < key <= max:
                sum += value[1]
            if key > max:
                break
        return sum
    
    

C:\Users\guner\AppData\Local\Temp\ipykernel_15712\868397739.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Dataset Generation Worflow

- We start from a graph randomly generated called start
- We define the a 'Consecutor' and 'ConsecutorExecutor'
- We use the latter to generate an history of changes
- We use HistoryUtilities to generate a df from the history combinations
- We save our df to disk

Following there are two function useful to create the datasets:
- The first is used to split a dataset into two portions
- The second is used to clean a folder of jsons files and store a dataset into sparse jsons (representing datapoints)

In [9]:
import pandas as pd
import os
import json

def split_by_fractions(df:pd.DataFrame, train=0.8, test=0.2, random_state:int=42):
    assert train+test==1.0, 'fractions sum is not 1.0'
    train = df.sample(frac=train, random_state=random_state)
    test = df.drop(train.index)
    train = train.reset_index(drop=True)
    test = test.reset_index(drop=True)
    return train, test

def save_to_sparse_json(df : pd.DataFrame, outfolder : str):
    # Clean Folder
    file_list = os.listdir(outfolder)
    json_files = [file for file in file_list if file.endswith('.json')]
    for file in json_files:
        file_path = os.path.join(outfolder, file)
        os.remove(file_path)
    # Save to Folder
    for index, row in df.iterrows():
        filename = os.path.join(outfolder, f'graph_{index}.json')
        with open(filename, 'w') as f:
            json.dump(row.to_dict(), f, indent=4)
            
def save_to_dense_json(df : pd.DataFrame, outpath : str):
    df.to_json(outpath, orient='records', default_handler=vars, lines=True, force_ascii=False, mode='w')

### Incremental Dataset Generation Example

In [10]:
# generator = RandomGraphGenerator()
# start = generator.generate_random_ER_graph()

# inc_consecutor = IncrementalConsecutor()
# exc_consecutor = ConsecutorExecutor(inc_consecutor)

# history = exc_consecutor.execute(start, steps=10)
# hist_utils = HistoryUtilities()
# hist_utils.assert_geds(history)

# plot_cons_hist_entries(history, stop=5)

# df = hist_utils.build_combinations_df(history)
# df = df.drop_duplicates(subset=['graph_1', 'graph_2', 'labels_1', 'labels_2'])
# df.to_json('./dataset/incremental.json', force_ascii=False, default_handler=vars, orient='records', lines=True)

### Decremental Dataset Generation Example

In [11]:
# generator = RandomGraphGenerator()
# start = generator.generate_random_ER_graph()

# dec_consecutor = DecrementalConsecutor()
# exc_consecutor = ConsecutorExecutor(dec_consecutor)

# history = exc_consecutor.execute(start, steps=10)
# hist_utils = HistoryUtilities()
# hist_utils.assert_geds(history)

# plot_cons_hist_entries(history, stop=5)

# df = hist_utils.build_combinations_df(history)
# df = df.drop_duplicates(subset=['graph_1', 'graph_2', 'labels_1', 'labels_2'])
# df.to_json('./dataset/decremental.json', force_ascii=False, default_handler=vars, orient='records', lines=True)

## Reverse GEDs checks just to be sure

Altough GEDs are asserted in the previous steps, I initially faced a problem causing the stored data not to be consistent with real GEDs.
This was due to the fact that edge matrix are stored to the disk hence isolated nodes information was lost. Now that cannot happen anymore and the problem is solved.

In [12]:
# import pandas as pd

# def reverse_assert_ged(df : pd.DataFrame):
#     g1s = df.graph_1.to_list()
#     g2s = df.graph_2.to_list()
#     geds = df.ged.to_list()

#     for i in range(len(g1s)):
#         G1 = nx.Graph(g1s[i])
#         G2 = nx.Graph(g2s[i])
#         ged = geds[i]
#         rged = nx.graph_edit_distance(G1, G2)
#         assert rged == ged

# df_inc = pd.read_json('./dataset/incremental.json', orient='records', lines=True)
# df_dec = pd.read_json('./dataset/decremental.json', orient='records', lines=True)
# reverse_assert_ged(df_inc)
# reverse_assert_ged(df_dec)

## Dataset Generation

Parameters used for EXTRASMALL:
RANGE = 25;
ER = 3,10,0.3,1;
BA = 3,10;
STEPS = 3,20

Parameters used for SMALL:
RANGE = 75;
ER = 3,15,0.3,1;
BA = 3,15;
STEPS = 5,30;

Parameters used for SMALLDENSE:
RANGE = 1000;
ER = 3,15,0.3,1;
BA = 3,15;
STEPS = 5,25;

Parameters used for MEDIUM:
RANGE = 150;
ER = 3,20,0.3,1;
BA = 3,20;
STEPS = 10,40;

Parameters used for MEDIUMDENSE:
RANGE = 1000;
ER = 3,20,0.3,1;
BA = 3,20;
STEPS = 10,40;

Parameters used for REFERENCE TESTSET:
RANGE = 1000;
ER = 3,100,0.3,1;
BA = 3,100;
STEPS = 1,1;

In [13]:
generator = RandomGraphGenerator()
hist_utils = HistoryUtilities()
dataset = pd.DataFrame()
stop_on_empty = lambda G: len(list(G.nodes))==0

# Change the range parameter to generate more data
for _ in range(250):
    rand = random()
    #change the parameters in the generator's function to generate bigger or smaller graphs
    if rand <= 0.6:
        start = generator.generate_random_ER_graph(3,10,0.3,1)
    else:
        start = generator.generate_random_BA_graph(3,10)
        
    rand = random()
    if rand <= 0.6:
        consecutor = IncrementalConsecutor()
    else:
        consecutor = DecrementalConsecutor()
        
    exc_consecutor = ConsecutorExecutor(consecutor)
    
    # Change the parameter to generate more consecutio steps
    rand = randint(3, 20)
    history = exc_consecutor.execute(start, steps=rand, stopper=stop_on_empty, skip_zero_ged=True)
    
    df = hist_utils.build_combinations_df(history)
    df = df.sample(frac=1).reset_index(drop=True)
    dataset = pd.concat([dataset, df], ignore_index=True)

dataset = dataset[~dataset.astype(str).duplicated(subset=['graph_1', 'graph_2', 'labels_1', 'labels_2'])]
dataset = dataset.sample(frac=1).reset_index(drop=True)
display(dataset)

train, test = split_by_fractions(dataset, train=0.9, test=0.1)
display(train)
display(test)

save_to_dense_json(train, 'train.json')
save_to_dense_json(test, 'test.json')

,graph_1,graph_2,ged,labels_1,labels_2
0,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",15.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[(0, 1), (0, 2), (0, 3), (0, 5), (0, 8), (0, 7...","[(0, 1), (0, 2), (0, 3), (0, 5), (0, 8), (0, 7...",10.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,"[(0, 5), (0, 6), (1, 5), (2, 6), (5, 6)]","[(2, 6)]",7.0,"[1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1]"
3,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",24.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,"[(0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 2...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 4...",8.0,"[1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...,...
12308,"[(0, 4), (1, 5), (1, 2), (1, 4), (3, 4), (3, 5)]","[(0, 4), (0, 2), (0, 3), (0, 6), (0, 7), (1, 5...",24.0,"[1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
12309,"[(0, 1), (0, 2), (0, 4), (0, 10), (0, 11), (0,...","[(0, 1), (0, 2), (0, 4), (0, 10), (0, 11), (0,...",12.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
12310,"[(0, 2), (0, 3), (0, 1), (0, 4), (0, 5), (1, 2...","[(0, 2), (0, 3), (0, 1), (0, 4), (0, 5), (0, 6...",14.0,"[1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]"
12311,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",40.0,"[1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


,graph_1,graph_2,ged,labels_1,labels_2
0,"[(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8...","[(0, 1), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8...",11.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",3.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,"[(0, 1), (0, 2), (0, 3), (0, 4), (1, 3), (1, 4...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 3...",12.0,"[1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1]"
3,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 3)]",19.0,"[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1]"
4,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",9.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...,...
11077,"[(0, 1), (0, 2), (0, 3), (0, 6), (1, 3), (1, 4...","[(0, 1), (0, 2), (0, 3), (0, 6), (0, 5), (1, 3...",2.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
11078,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 7...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 7...",21.0,"[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
11079,"[(0, 2), (0, 3), (0, 5), (0, 9), (1, 8), (2, 7...","[(0, 2), (0, 3), (0, 5), (0, 9), (0, 10), (0, ...",40.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
11080,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",16.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


,graph_1,graph_2,ged,labels_1,labels_2
0,"[(0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 2...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 4...",8.0,"[1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]"
1,"[(1, 2), (1, 3), (1, 5), (1, 6), (2, 4), (2, 5...","[(1, 6)]",27.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1]"
2,"[(0, 2), (0, 3), (0, 5), (0, 7), (0, 4), (1, 5...","[(0, 2), (0, 3), (0, 5), (0, 7), (0, 4), (0, 1...",9.0,"[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 3)]",14.0,"[1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1]"
4,"[(0, 1), (0, 4), (0, 7), (1, 2), (1, 4), (1, 5...","[(0, 1), (0, 4), (0, 7), (0, 6), (0, 2), (0, 9...",15.0,"[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...,...
1226,"[(0, 2), (0, 5), (2, 5)]","[(2, 5)]",3.0,"[1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1]"
1227,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",3.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1228,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",4.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1229,"[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...","[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6...",12.0,"[1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
